In [1]:
from pyspark.sql import SparkSession 

sc = SparkSession.builder.appName("DataFrame").config("spark.driver.memory", "8g").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/22 01:26:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
logs_interesse = [
    "Eleitor foi habilitado",
    "Voto confirmado para [Deputado Federal]",
    "Voto confirmado para [Deputado Estadual]",
    "Voto confirmado para [Senador]",
    "Voto confirmado para [Governador]",
    "Voto confirmado para [Presidente]",
    "Tecla indevida pressionada",
]
regex_interesse = [
    "Município:",
    "Zona Eleitoral:",
    "Local de Votação:",
    "Seção Eleitoral:",
]
regex_interesse = "|".join(regex_interesse)

In [3]:
import pyspark.sql.functions as F

DATA_PATH = "/app/data/"
input_file = DATA_PATH+"decodificado/*" 

source = sc.read.csv(input_file, sep="\t", header=False)
source = source.withColumn("arquivo", F.input_file_name())

In [4]:
"""distinto = source.select('arquivo').distinct()
amostra = distinto.sample(False, (100/distinto.count()),seed=1100)
amostra = amostra.toPandas()['arquivo'].values
source =  source.filter(F.col("arquivo").isin(*amostra))
print(f'Número de urnas selecionadas {len(amostra)}')"""

'distinto = source.select(\'arquivo\').distinct()\namostra = distinto.sample(False, (100/distinto.count()),seed=1100)\namostra = amostra.toPandas()[\'arquivo\'].values\nsource =  source.filter(F.col("arquivo").isin(*amostra))\nprint(f\'Número de urnas selecionadas {len(amostra)}\')'

In [5]:
df = source.withColumnRenamed("_c0", "data").withColumnRenamed("_c4", "descricao")
df = df.select("data", "descricao")
df = df.withColumn("arquivo", F.input_file_name())
df = df.filter(
    F.col("descricao").isin(logs_interesse) | F.col("descricao").rlike(regex_interesse)
)

df = df.withColumn("data", F.to_timestamp(F.col("data"), "dd/MM/yyyy HH:mm:ss"))
df.show(truncate=False)

+-------------------+----------------------+------------------------------------------------------+
|data               |descricao             |arquivo                                               |
+-------------------+----------------------+------------------------------------------------------+
|2022-09-22 09:36:53|Município: 16039      |file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:36:53|Zona Eleitoral: 0029  |file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:36:53|Local de Votação: 1082|file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:36:53|Seção Eleitoral: 0128 |file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:41:29|Município: 16039      |file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:41:29|Zona Eleitoral: 0029  |file:///app/data/decodificado/o00406-1603900290128.txt|
|2022-09-22 09:41:29|Seção Eleitoral: 0128 |file:///app/data/decodificado/o00406-1603900290128.txt|


In [6]:
from pyspark.sql.window import Window
from unidecode import unidecode

window = (
    Window.orderBy("data")
    .partitionBy("arquivo") #Paraleliza pelo arquivo
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

df_window = df
for nome_extracao in ["Município", "Zona Eleitoral", "Seção Eleitoral"]:
    nome_col = unidecode(nome_extracao).replace(" ", "_")
    df_window = df_window.withColumn(
        nome_col,
        F.when(
            F.col("descricao").contains(nome_extracao),
            F.regexp_extract(F.col("descricao"), rf"{nome_extracao}: (\d+)", 1).cast(
                "integer"
            ),
        ).otherwise(None),
    )

    df_window = df_window.withColumn(
        nome_col, F.last(nome_col, ignorenulls=True).over(window)
    )
df_window.show(truncate=False)

+-------------------+----------------------+------------------------------------------------------+---------+--------------+---------------+
|data               |descricao             |arquivo                                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|
+-------------------+----------------------+------------------------------------------------------+---------+--------------+---------------+
|2022-09-20 16:44:40|Município: 16004      |file:///app/data/decodificado/o00406-1600400210017.txt|16004    |NULL          |NULL           |
|2022-09-20 16:44:40|Zona Eleitoral: 0021  |file:///app/data/decodificado/o00406-1600400210017.txt|16004    |21            |NULL           |
|2022-09-20 16:44:40|Local de Votação: 1015|file:///app/data/decodificado/o00406-1600400210017.txt|16004    |21            |NULL           |
|2022-09-20 16:44:40|Seção Eleitoral: 0017 |file:///app/data/decodificado/o00406-1600400210017.txt|16004    |21            |17             |
|2022-09-20 1

In [7]:
df_teclas = df_window.filter(~F.col("descricao").rlike(regex_interesse))
df_teclas = df_teclas.withColumn(
    "erros_tecla",
    F.when(F.col("descricao") == "Tecla indevida pressionada", 1).otherwise(0),
)

df_teclas = df_teclas.withColumn(
    "id_eleitor", F.when(F.col("descricao") == "Eleitor foi habilitado", 1).otherwise(0)
)
my_window = (
    Window.orderBy("data")
    .partitionBy("arquivo")
    .rowsBetween(Window.unboundedPreceding, 0)
)
df_teclas = df_teclas.withColumn("id_eleitor", F.sum("id_eleitor").over(my_window))


df_teclas.orderBy(["arquivo","data"], ascending=[1,1]).show(truncate=False)

+-------------------+----------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+
|data               |descricao                               |arquivo                                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|
+-------------------+----------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+
|2022-10-02 08:01:01|Eleitor foi habilitado                  |file:///app/data/decodificado/o00406-1600400210016.txt|16004    |21            |16             |0          |1         |
|2022-10-02 08:01:29|Voto confirmado para [Deputado Federal] |file:///app/data/decodificado/o00406-1600400210016.txt|16004    |21            |16             |0          |1         |
|2022-10-02 08:01:47|Voto confirmado para [Deputado Estadual]|file:///app/data/decodificad

In [8]:
my_window = (
    Window.orderBy("data")
    .partitionBy("arquivo","id_eleitor")
    .rowsBetween(Window.unboundedPreceding, 0)
)
df_soma_ate_momento = df_teclas.withColumn("erros_ate_agora", F.sum("erros_tecla").over(my_window))
df_soma_ate_momento.show()

+-------------------+--------------------+--------------------+---------+--------------+---------------+-----------+----------+---------------+
|               data|           descricao|             arquivo|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|erros_ate_agora|
+-------------------+--------------------+--------------------+---------+--------------+---------------+-----------+----------+---------------+
|2022-10-02 08:05:03|Eleitor foi habil...|file:///app/data/...|    16004|            21|             17|          0|         1|              0|
|2022-10-02 08:05:27|Voto confirmado p...|file:///app/data/...|    16004|            21|             17|          0|         1|              0|
|2022-10-02 08:05:38|Tecla indevida pr...|file:///app/data/...|    16004|            21|             17|          1|         1|              1|
|2022-10-02 08:05:42|Voto confirmado p...|file:///app/data/...|    16004|            21|             17|          0|         1|         

In [9]:
agrupado_eleitor = df_soma_ate_momento.groupBy("arquivo", "id_eleitor").agg({"erros_tecla": "sum"}).orderBy(["arquivo","id_eleitor"], ascending=[1,1])
agrupado_eleitor.show(truncate=False)

+------------------------------------------------------+----------+----------------+
|arquivo                                               |id_eleitor|sum(erros_tecla)|
+------------------------------------------------------+----------+----------------+
|file:///app/data/decodificado/o00406-1600400210016.txt|1         |0               |
|file:///app/data/decodificado/o00406-1600400210016.txt|2         |0               |
|file:///app/data/decodificado/o00406-1600400210016.txt|3         |1               |
|file:///app/data/decodificado/o00406-1600400210016.txt|4         |7               |
|file:///app/data/decodificado/o00406-1600400210016.txt|5         |6               |
|file:///app/data/decodificado/o00406-1600400210016.txt|6         |26              |
|file:///app/data/decodificado/o00406-1600400210016.txt|7         |0               |
|file:///app/data/decodificado/o00406-1600400210016.txt|8         |0               |
|file:///app/data/decodificado/o00406-1600400210016.txt|9        

In [10]:
agrupado_eleitor.orderBy(["sum(erros_tecla)"], ascending=[0]).show(truncate=False)

+------------------------------------------------------+----------+----------------+
|arquivo                                               |id_eleitor|sum(erros_tecla)|
+------------------------------------------------------+----------+----------------+
|file:///app/data/decodificado/o00406-1781700120175.txt|22        |482             |
|file:///app/data/decodificado/o00406-1853800270078.txt|5         |382             |
|file:///app/data/decodificado/o00406-1759000330275.txt|48        |358             |
|file:///app/data/decodificado/o00406-1683700490080.txt|173       |355             |
|file:///app/data/decodificado/o00406-1618700430062.txt|21        |337             |
|file:///app/data/decodificado/o00406-1605500180059.txt|44        |320             |
|file:///app/data/decodificado/o00406-1889900490110.txt|71        |309             |
|file:///app/data/decodificado/o00406-1747700380021.txt|155       |289             |
|file:///app/data/decodificado/o00406-1787600400132.txt|10       

In [12]:
teste = df_soma_ate_momento.filter((F.abs(F.col("id_eleitor") - 22) < 1) & (F.col("arquivo") == "file:///app/data/decodificado/o00406-1781700120175.txt"))
teste.show(100,truncate=False)


+-------------------+---------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+---------------+
|data               |descricao                              |arquivo                                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|erros_ate_agora|
+-------------------+---------------------------------------+------------------------------------------------------+---------+--------------+---------------+-----------+----------+---------------+
|2022-10-02 09:10:36|Eleitor foi habilitado                 |file:///app/data/decodificado/o00406-1781700120175.txt|17817    |12            |175            |0          |22        |0              |
|2022-10-02 09:10:54|Tecla indevida pressionada             |file:///app/data/decodificado/o00406-1781700120175.txt|17817    |12            |175            |1          |22        |1              |
|2022-10-02 09:

In [11]:
df_soma_tecla = df_soma_ate_momento.join(agrupado_eleitor, on=['arquivo', 'id_eleitor'], how='left')
df_soma_tecla = df_soma_tecla.drop("erros_tecla")
df_soma_tecla.show(5)

+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+
|             arquivo|id_eleitor|               data|           descricao|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_ate_agora|sum(erros_tecla)|
+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+
|file:///app/data/...|         1|2022-10-02 08:03:11|Eleitor foi habil...|    16004|            21|             53|              0|               2|
|file:///app/data/...|         1|2022-10-02 08:03:20|Voto confirmado p...|    16004|            21|             53|              0|               2|
|file:///app/data/...|         1|2022-10-02 08:03:28|Tecla indevida pr...|    16004|            21|             53|              1|               2|
|file:///app/data/...|         1|2022-10-02 08:03:32|Voto confirmado p...|    16004|            21|       

In [12]:
df_sem_tecla = df_soma_tecla.filter(
    ~F.col("descricao").isin(["Tecla indevida pressionada"])
)
df_sem_tecla.show(5)

+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+
|             arquivo|id_eleitor|               data|           descricao|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_ate_agora|sum(erros_tecla)|
+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+
|file:///app/data/...|         1|2022-10-02 08:03:11|Eleitor foi habil...|    16004|            21|             53|              0|               2|
|file:///app/data/...|         1|2022-10-02 08:03:20|Voto confirmado p...|    16004|            21|             53|              0|               2|
|file:///app/data/...|         1|2022-10-02 08:03:32|Voto confirmado p...|    16004|            21|             53|              1|               2|
|file:///app/data/...|         1|2022-10-02 08:03:39|Voto confirmado p...|    16004|            21|       

In [13]:
w = Window.partitionBy("arquivo", "id_eleitor").orderBy("data")
df_tempo_segundos = (
    df_sem_tecla.withColumn("lag_tempo", F.lag(df_teclas["data"], 1).over(w))
    .withColumn(
        "tempo(segundos)",
        (
            F.unix_timestamp(
                df_teclas["data"],
            )
            - F.unix_timestamp(F.col("lag_tempo"))
        ),
    )
)
df_tempo_segundos.show()

+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+-------------------+---------------+
|             arquivo|id_eleitor|               data|           descricao|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_ate_agora|sum(erros_tecla)|          lag_tempo|tempo(segundos)|
+--------------------+----------+-------------------+--------------------+---------+--------------+---------------+---------------+----------------+-------------------+---------------+
|file:///app/data/...|         9|2022-10-02 08:37:01|Eleitor foi habil...|    16004|            21|             16|              0|               3|               NULL|           NULL|
|file:///app/data/...|         9|2022-10-02 08:37:39|Voto confirmado p...|    16004|            21|             16|              0|               3|2022-10-02 08:37:01|             38|
|file:///app/data/...|         9|2022-10-02 08:37:55|Voto confirmado p...| 

In [14]:
df_final = df_tempo_segundos.drop("lag_tempo")
df_final = df_final.filter(
    ~F.col("descricao").isin(["Eleitor foi habilitado"])
)
df_final = df_final.withColumn("descricao", F.regexp_replace("descricao", r"(\[|\]|\bVoto confirmado para\b)", ""))
df_final.show()

+--------------------+----------+-------------------+------------------+---------+--------------+---------------+---------------+----------------+---------------+
|             arquivo|id_eleitor|               data|         descricao|Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_ate_agora|sum(erros_tecla)|tempo(segundos)|
+--------------------+----------+-------------------+------------------+---------+--------------+---------------+---------------+----------------+---------------+
|file:///app/data/...|         9|2022-10-02 08:37:39|  Deputado Federal|    16004|            21|             16|              0|               3|             38|
|file:///app/data/...|         9|2022-10-02 08:37:55| Deputado Estadual|    16004|            21|             16|              0|               3|             16|
|file:///app/data/...|         9|2022-10-02 08:38:09|           Senador|    16004|            21|             16|              0|               3|             14|
|file:///app/data/...|

In [15]:
df_final.write.mode('overwrite').parquet(DATA_PATH+"dados_log_urna")

In [13]:
sc.stop()